In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import datetime as dt
import alpaca_trade_api as tradeapi
import json
from alpaca_trade_api.rest import REST, TimeFrame
load_dotenv()

True

In [2]:
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

tradeapi = REST(alpaca_key, alpaca_secret_key)

In [3]:
def get_daily_trade_data(ticker, start, end, tradeapi): 
    ticker_df = tradeapi.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
    ticker_df['ticker'] = ticker

    return ticker_df

In [4]:

# get_daily_trade_data('HD', '2022-01-03', '2022-01-07', tradeapi)

In [5]:
ticker_list = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

def make_tickers_df(ticker_list, start_date_str, end_date_str, tradeapi):
    ticker_dfs_list = [get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list]
    tickers_df = pd.concat(ticker_dfs_list, axis=0, join='outer')
    tickers_df.index = tickers_df.index.date
    tickers_df = tickers_df[['ticker','close','volume']]
    return tickers_df
    
    #['symbol','close'].reset_index().rename(columns={'timestamp':'date'}).set_index('date').dt.date
    #return tickers_df.index.dt.date.reset_index().rename(columns={"timestamp":"date"})

In [6]:
# stocks_df = make_tickers_df(ticker_list, '2012-06-01', '2022-06-01', tradeapi)
# stocks_df.head()

In [7]:
def make_tickers_dict(ticker_list, start_date_str, end_date_str, tradeapi):
    return {ticker: get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list}

In [8]:
# ticker_list = ['AAPL', 'XOM']
# my_tick_dict = make_tickers_dict(ticker_list, '2022-01-03', '2022-01-07', tradeapi)
# my_tick_dict

## News Article Vader Analyzer

In [9]:
from libs.drew_lib import *

from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
api_key = os.getenv("NEWSAPI_KEY")
newsapi = NewsApiClient(api_key=api_key)
print(type(api_key))

<class 'str'>


In [11]:
# company_df = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')
# company_df = company_df.rename(columns={'Company Name': 'Company_Name'})
# company_dict = dict(zip(company_df.Company_Name, company_df.Ticker))
# company_list = list(company_dict.keys())
# tickers_list = company_dict.values()
# ls = [type(item) for item in company_list]
# print(ls)
# tickers_list


In [12]:
nflx_news_df = form_df('Netflix')
nflx_news_df

,date,ticker,text
0,2022-05-19,NFLX,Picking out something new to watch from Netfli...
1,2022-05-14,NFLX,The platform could add the capability for come...
2,2022-06-02,NFLX,The streaming wars are no longer about which a...
3,2022-05-10,NFLX,Streaming is a curious beast. One minute you'l...
4,2022-05-10,NFLX,The timetables are accelerating\r\nIllustratio...
5,2022-05-26,NFLX,Aloy could be on her way to Netflix.\r\n\n \n\...
6,2022-05-22,NFLX,
7,2022-05-28,NFLX,"Over a three-hour dinner, Mr. Sarandos was cha..."
8,2022-05-08,NFLX,"Ncuti Gatwa, a star of the Netflix series Sex ..."
9,2022-05-18,NFLX,Alongside 150 full time employees Netflix axed...


In [13]:
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
# def vader(df):
#     company_list = company_dict.keys()

#     df_list = []
#     for company in company_list:
#         try:
#             df_list.append(form_df(company))
#         except:
#             continue
#     news_df = pd.concat(df_list, axis=0, join='outer')
    
#     return news_df



In [15]:
def vader_analyzer(df):
    analyzer = SentimentIntensityAnalyzer()
    df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]
    df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['text']]
    df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['text']]
    df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['text']]

    df['date'] = pd.to_datetime(
    df['date'],
    infer_datetime_format = True,
    utc = True    
    )
    df['date'] = df['date'].dt.date
    
    return df



In [16]:

# def daily_sentiment(df):
#     df = vader_analyzer(df)

#     df['date'] = pd.to_datetime(
#     df['date'],
#     infer_datetime_format = True,
#     utc = True    
#     )
    
#     return df.groupby('date')['compound','pos','neu','neg'].mean()


# # date to datetime

#     df['date'] = pd.to_datetime(
#     df['date'],
#     infer_datetime_format = True,
#     utc = True    
#     )
#     df['date'] = df['date'].dt.date


In [17]:
# df = vader_analyzer(netflix_news_df)
# df

In [18]:
news_df_list = [form_df(company) for company in company_list]
vader_news_df_list = [vader_analyzer(df) for df in news_df_list]
vader_news_df = pd.concat(vader_news_df_list, axis=0, join='inner')
news_sent_avg = vader_news_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
news_sent_avg = news_sent_avg[['date','ticker','pos','neg','neu','compound']]


news_sent_avg.to_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')

C:\Users\altma\AppData\Local\Temp\ipykernel_10048\3887123157.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  news_sent_avg = vader_news_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()


In [19]:
# perform sentiment analysis and find average sentiment by date

def daily_sentiment(df):
    vader_df = vader_analyzer(df)
    vader_df = vader_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
    vader_df = vader_df[['date','ticker','pos','neg','neu','compound']]
    return vader_df

## Helper Functions for PuttingItTogether

In [20]:
# function to filter through all csv files in Cleaned_Data and return dataframe for a ticker

def stock_picker(ticker):

    stock_df_list = []
    file_path = '../Notebooks/Data/Cleaned_Data'

    for filename in os.listdir(file_path):
        if filename.endswith(".csv") and filename != 'Ticker_library.csv' or 'news_sentiment.csv':
            csv_df = pd.read_csv(file_path +'/'+ filename, parse_dates=True, infer_datetime_format=True,index_col='datetime')
            #csv_df = csv_df.loc[csv_df['ticker'] == ticker]
            #csv_df.drop(columns='ticker',axis=1,inplace=True)
            #csv_df.set_index('date',inplace=True)
            stock_df_list.append(csv_df)
    all_ticker_data_df = pd.concat(stock_df_list,axis=1,join='outer')
    #all_ticker_data_df.insert(0, 'ticker', ticker)
    return all_ticker_data_df.drop(columns='Unnamed: 0')


In [21]:
def return_bin(df):

    bins = [-1000, -0.08, -0.03, -0.0000001, 0.0000001, 0.03, 0.08, 1000]
    labels = ['large loss','medium loss','small loss','no gain/loss','small gain','medium gain','large gain']
    
    df['returns'] = df['close'].pct_change()
    df['return_bin'] = pd.cut(df['returns'], bins=bins, labels=labels)

    return df

## Data prep for model

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [30]:
aapl_sent_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/apple_sentiment.csv', infer_datetime_format=True, parse_dates=True,index_col='datetime')
aapl_sent_df = aapl_sent_df.fillna(0)
aapl_sent_df

,articles_compound_sentiment,articles_positive_sentiment,articles_neutral_sentiment,articles_negative_sentiment,stockmarket_compound_sentiment,stockmarket_positive_sentiment,stockmarket_neutral_sentiment,stockmarket_negative_sentiment,securityanalysis_compound_sentiment,securityanalysis_positive_sentiment,securityanalysis_neutral_sentiment,securityanalysis_negative_sentiment,algotrading_compound_sentiment,algotrading_positive_sentiment,algotrading_neutral_sentiment,algotrading_negative_sentiment,wallstreetbets_compound_sentiment,wallstreetbets_positive_sentiment,wallstreetbets_neutral_sentiment,wallstreetbets_negative_sentiment
datetime,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.000000,0.00000,0.0000,0.00000,-0.319520,0.045600,0.838600,0.115800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-02,0.000000,0.00000,0.0000,0.00000,0.252993,0.070267,0.919000,0.010667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-03,0.000000,0.00000,0.0000,0.00000,0.212671,0.106000,0.815857,0.078000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-04,0.000000,0.00000,0.0000,0.00000,0.573991,0.184636,0.744364,0.071000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-05,0.000000,0.00000,0.0000,0.00000,0.033250,0.066500,0.860500,0.073000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-02-11,0.000000,0.00000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-02-12,0.000000,0.00000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-02-13,0.000000,0.00000,0.0000,0.00000,0.105544,0.074563,0.847313,0.078125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
stock_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/stock_data.csv',infer_datetime_format=True,parse_dates=True)
stock_df = stock_df.loc[stock_df['ticker']=='AAPL']
stock_df = stock_df.set_index('date').drop(columns=['ticker'])
stock_df
#model_df = pd.concat([aapl_sent_df,stock_df],axis=1,join='outer')

,close,volume
date,,


In [ ]:
nflx_df = stock_picker('NFLX')
#nflx_df = return_bin(nflx_df)
#nflx_df = nflx_df[nflx_df['return_bin'].notna()].fillna(0)

nflx_df.head()

In [ ]:
X = nflx_df.copy()
X = nflx_df.drop(columns=['return_bin','ticker'])
y = nflx_df['return_bin'].values
y = y.reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
X_scaler = StandardScaler()
X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
enc = OneHotEncoder()
enc.fit(y_train)

y_train_enc = enc.transform(y_train).toarray()
y_test_enc = enc.transform(y_test).toarray()

In [ ]:
model = Sequential()

number_units = 30
dropout_fraction = 0.2

In [ ]:
# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train_scaled.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Train the model
model.fit(X_train_scaled, y_train_enc, epochs=20, shuffle=False, batch_size=90, verbose=1)

# Evaluate the model
model.evaluate(X_test_scaled, y_test_enc, verbose=0)

# Make predictions using the testing data X_test
predicted = model.predict(X_test_scaled)

Epoch 1/20
14/14 [==============================] - 3s 7ms/step - loss: nan
Epoch 2/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 3/20
14/14 [==============================] - 0s 8ms/step - loss: nan
Epoch 4/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 5/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 6/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 7/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 8/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 9/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 10/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 11/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 12/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 13/20
14/14 [==============================] - 0s 6ms/step - loss: nan
Epoch 14

In [ ]:
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.activity.values,
    "Predicted": predicted
})
results.head(10)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').